Suggestion from Claude to analyze Results after training

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import sys
import numpy as np
import pandas as pd 

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from trading.agents.training_agent import TrainingStats

def analyze_training_results():
    # Load saved stats
    stats = TrainingStats()
    stats.load_from_disk("final_training_stats.json")

    # 1. Plot overall training metrics
    stats.plot_metrics("training_analysis.png")

    # 2. Calculate key performance metrics
    final_stats = stats.get_recent_stats(window=100)
    print("\nFinal Performance Metrics:")
    print(f"Average Reward: {final_stats['avg_reward']:.2f}")
    print(f"Win Rate: {final_stats['avg_win_rate']:.2%}")
    print(f"Average Trades per Episode: {final_stats['avg_trades']:.1f}")
    print(f"Average Drawdown: {final_stats['avg_drawdown']:.2%}")

    # 3. Analyze trading behavior
    trades_analysis = pd.DataFrame({
        'trades': stats.trade_counts,
        'win_rate': stats.win_rates,
        'duration': stats.trade_durations
    })
    
    print("\nTrading Behavior Analysis:")
    print(trades_analysis.describe())

    # 4. Calculate risk metrics
    risk_metrics = {
        'max_drawdown': max(stats.drawdowns),
        'avg_drawdown': np.mean(stats.drawdowns),
        'drawdown_std': np.std(stats.drawdowns),
        'worst_episode': min(stats.episode_rewards),
        'best_episode': max(stats.episode_rewards)
    }
    
    print("\nRisk Metrics:")
    for metric, value in risk_metrics.items():
        print(f"{metric}: {value:.4f}")

    # 5. Position analysis
    position_df = pd.DataFrame(stats.position_ratios)
    print("\nPosition Distribution:")
    print(position_df.mean())  # Average position ratios

    # 6. Learning progression
    learning_metrics = pd.DataFrame({
        'policy_loss': stats.losses['policy_loss'],
        'value_loss': stats.losses['value_loss'],
        'entropy_loss': stats.losses['entropy_loss'],
        'learning_rate': stats.learning_rates
    })
    
    print("\nLearning Progression (last 1000 steps):")
    print(learning_metrics.tail(1000).describe())

    # 7. Create detailed performance report
    report = f"""
    Training Performance Report
    =========================
    Training Duration: {(stats.timestamps[-1] - stats.timestamps[0]).total_seconds() / 3600:.1f} hours
    Total Episodes: {len(stats.episode_rewards)}
    
    Final Performance (last 100 episodes):
    - Average Reward: {final_stats['avg_reward']:.2f}
    - Win Rate: {final_stats['avg_win_rate']:.2%}
    - Average Trades: {final_stats['avg_trades']:.1f}
    - Average Drawdown: {final_stats['avg_drawdown']:.2%}
    
    Risk Metrics:
    - Maximum Drawdown: {risk_metrics['max_drawdown']:.2%}
    - Average Drawdown: {risk_metrics['avg_drawdown']:.2%}
    - Best Episode: {risk_metrics['best_episode']:.2f}
    - Worst Episode: {risk_metrics['worst_episode']:.2f}
    
    Trading Behavior:
    - Average Position Duration: {np.mean(stats.trade_durations):.1f} steps
    - Average Trades per Episode: {np.mean(stats.trade_counts):.1f}
    """

    # Save report
    with open("training_report.txt", "w") as f:
        f.write(report)

    return trades_analysis, risk_metrics, position_df, learning_metrics

# Usage:
trades_analysis, risk_metrics, position_df, learning_metrics = analyze_training_results()

# Optional: Create interactive visualizations
import plotly.express as px
import plotly.graph_objects as go

def create_interactive_dashboard(stats):
    # Create interactive plots
    fig = go.Figure()
    
    # Add balance progression
    fig.add_trace(go.Scatter(
        y=stats.balances,
        name='Account Balance'
    ))
    
    # Add drawdown
    fig.add_trace(go.Scatter(
        y=stats.drawdowns,
        name='Drawdown',
        yaxis='y2'
    ))
    
    fig.update_layout(
        title='Trading Performance Over Time',
        yaxis2=dict(
            title='Drawdown',
            overlaying='y',
            side='right'
        )
    )
    
    # Save interactive dashboard
    fig.write_html("trading_dashboard.html")

FileNotFoundError: [Errno 2] No such file or directory: 'training_stats/final_training_stats.json'